# Create dataset translation to Python3 using class

In [ ]:
"""
Created on Mon May  2 05:06:26 2016

@author: diogo

@modification: Déruaz Vincent
"""
import sys
import pickle as cPickle # import module first
# import pylab as plt

import matplotlib as mpl

mpl.use('Agg')
# import matplotlib.pyplot as plt
import numpy as np
import csv
# from __future__ import division
import os.path

class dataset_generator():
    def __init__(self, path_to_config, path_save_file, type_of_bins, bins, dataset_type):
        self.path_to_config = path_to_config
        self.path_save_file = path_save_file
        #Number of Bins (1) or Vector of Bins (2)
        self.type_of_bins = type_of_bins
        #if (1) --> Number of Bins (1-90)
        #if (2) --> Quantity of Bins (1-90)
        self.bins = bins
        #Use normal dataSet : (Yes = 1/ No = 0)
        self.dataset_type = dataset_type

    def createQtdScoreNoNormal(self, verbose):
        interIdPreced = self.vecId[0]
        vecResultFinal = []
        vecResultsProv = []
        position = 0

        for idInter in self.vecId:
            #print('vecResultsProv: %d; vecResultFinal: %d' % (len(vecResultsProv), len(vecResultFinal)))
            #print(sys.getsizeof(vecResultFinal))
            sys.stdout.write('\r%d Mo' % (sys.getsizeof(vecResultFinal)))
            
            if interIdPreced == idInter:
                qtdScores = self.vecQtd[position]
                aux = 0
                while aux < qtdScores:
                    #print(self.vecScor[position])
                    vecResultsProv.append(self.vecScor[position])
                    aux = aux + 1

            else:
                self.vecIdsDS.append(interIdPreced)
                self.vecTypeClass.append(self.vecCla[position])

                interIdPreced = idInter
                if verbose:
                    print('position: %d' % (position))
                vecResultFinal.append(vecResultsProv)
                vecResultsProv = []

                qtdScores = self.vecQtd[position]
                aux = 0
                while aux < qtdScores:
                    vecResultsProv.append(self.vecScor[position])
                    aux = aux + 1
            position = position + 1

        print('\n')
        return vecResultFinal


    def createQtdScoreNormal(self, matValues):
        matNormalValues = []
        for interact in matValues:
            aux = len(interact)
            print("Nb Interaction: " + str(aux))
            vecNormalScore = []
            for valueInter in interact:
                auxScor = valueInter / aux
                vecNormalScore.append(auxScor)
            matNormalValues.append(vecNormalScore)
        return matNormalValues


    def createVecSize(self, matValues):
        qtdInteractions = []
        for nmInteractions in matValues:
            qtdInteractions.append(len(nmInteractions))
        return qtdInteractions


    def controlNumberInteractionsOnlyOne(self, vecListQtdInter, position):
        QtdInteractionInt = []
        onlyZeros = 0
        for i in vecListQtdInter:
            QtdInteractionInt.append(int(i))
            if i == self.qtdInteractionsAll[position]:
                onlyZeros = 1
        if onlyZeros == 1 and self.typeConfig == 1:
            QtdInteractionInt = [0] * (len(vecListQtdInter) - 3)
            QtdInteractionInt.insert(0, vecListQtdInter[0])
            QtdInteractionInt.insert(1, vecListQtdInter[1])
            QtdInteractionInt.insert(2, self.qtdInteractionsAll[position])
        return QtdInteractionInt


    def writeFile(self, n, binsList, vecid, vecCla, pathFile, boolNormaliz):
        # binsList = bins.tolist()
        position = 0
        binsListR = [round(i, 5) for i in binsList]

        if boolNormaliz == 1:
            binsListR = [format(i, '.8f') for i in binsListR]

        binsListR.insert(0, "ID_interaction")
        binsListR.insert(1, "Class_interactions")

        with open(pathFile, 'w') as outcsv:
            # configure writer to write standard csv file
            writer = csv.writer(outcsv, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
            writer.writerow(binsListR)
            for idInteract, typeClass, listHisto in zip(vecid, self.vecCla, n):
                listAux = listHisto.tolist()
                listAux.insert(0, typeClass)
                listAux.insert(0, idInteract)
                # Write item to outcsv
                # listAuxB = [int(i) for i in listAux]
                listAuxB = self.controlNumberInteractionsOnlyOne(listAux, position)
                writer.writerow(listAuxB)
                position = position + 1


    def creatVecNBins(self, matriceScores, Bins):
        matResultsBins = []
        matBinsNum = []
        print('matriceScores: %d' % (len(matriceScores)))
        for interaction in matriceScores:
            a, b = np.histogram(interaction, bins=Bins)
            matResultsBins.append(a)
            matBinsNum.append(b)
        return matResultsBins, matBinsNum


    # CONFIGR USER#################################################################
    def getUserConfigTypeDataSet(self):
        aux = 1
        x = -1
        while (aux == 1):
            try:
                x = self.dataset_type
            except:
                print("Input not legal")
            if (x == 1 or x == 0):
                aux = 0
        return x


    def getUserConfigBinds(self):
        aux = 1
        x = -1
        while (aux == 1):
            try:
                x = self.type_of_bins
            except:
                print("Input not legal")
            if (x == 1 or x == 2):
                aux = 0
        if (x == 1):
            aux = 1
            while (aux == 1):
                try:
                    x = self.bins
                except:
                    print("Input not legal")
                if (x > 0 and x < 91):
                    aux = 0
            return x, 1
        if (x == 2):
            aux = 1
            while (aux == 1):
                try:
                    x = self.bins
                except:
                    print("input not legal")
                if (x > 0 and x < 91):
                    aux = 0
            vecBins = []
            valueBin = -1
            while (aux < x):
                try:
                    valueBin = int(input("Insert value : "))
                except:
                    print("input not legal")
                    valueBin = -1
                if (valueBin >= 0):
                    vecBins.append(valueBin)
                    aux = aux + 1
            return vecBins, 2


    def pathFilePickle(self):
        aux = 0
        while (aux == 0):
            result = os.path.isfile(self.path_to_config)
            if (result == True):
                return self.path_to_config
            else:
                print("File not exists")


    def pathSaveFile(self):
        filePath = input("Save file: ")
        return filePath


    def compute(self):
        pathFile = self.pathFilePickle()

        #pathFileSave = pathSaveFile()

        # chemain = '/home/diogo/Bureau/gradesdict.p'

        # pathFileSave = '/home/diogo/Bureau/test.csv'

        f = open(pathFile, 'rb')  # 'r' for reading; can be omitted
        self.vecId, self.vecCla, self.vecScor, self.vecQtd = cPickle.load(f)  # dump data to f
        f.close()

        self.vecId
        self.vecIdsDS = []
        self.vecTypeClass = []

        vecQtdProteins = []

        vecResultFinal = []
        vecResultNormal = []

        binsConfig, self.typeConfig = self.getUserConfigBinds()

        typeDataSet = self.getUserConfigTypeDataSet()


        vecResultFinal = self.createQtdScoreNoNormal(False)


        self.qtdInteractionsAll = self.createVecSize(vecResultFinal)

        if (typeDataSet == 1):
            vecResultNormal = self.createQtdScoreNormal(vecResultFinal)

        qtdScoresHisto = []
        SepHisto = []

        print(binsConfig)

        if (typeDataSet == 1):
            qtdScoresHisto, SepHisto = self.creatVecNBins(vecResultNormal, binsConfig)
        else:
            qtdScoresHisto, SepHisto = self.creatVecNBins(vecResultFinal, binsConfig)

        len(vecResultFinal[0])

        vecNumBins = len(qtdScoresHisto[0])
        aux = 0
        auxVec = []
        while (aux < vecNumBins):
            auxVec.append(aux)
            aux = aux + 1



        self.writeFile(qtdScoresHisto, auxVec, self.vecIdsDS, self.vecTypeClass, self.path_save_file, 0)

# Simple computation with for demonstration

In [ ]:

dataset_generator_obj = dataset_generator(path_to_config = 'dataset/gradesdict.p',
                                      path_save_file = 'dataset_output/test.csv', 
                                      type_of_bins = 1, 
                                      bins = 19, 
                                      dataset_type = 0)
dataset_generator_obj.compute()

# Generation of more datasets

In [ ]:
for index in range(18,19):
    print('Dataset of %d bins' % (index))
    save_path = 'dataset_output/crap_dataset_%d_bins.csv' % (index)
    
    dataset_generator_obj = dataset_generator(path_to_config = 'dataset/gradesdict.p',
                                      path_save_file = save_path, 
                                      type_of_bins = 1, 
                                      bins = index, 
                                      dataset_type = 0)
    dataset_generator_obj.compute()